In [120]:
import os
import numpy as np 
import pandas as pd 
from itertools import islice, cycle
from more_itertools import pairwise
import scipy.sparse as sp
from itertools import islice, cycle
#from tqdm.auto import tqdm

# Данные

Изучаем данные

In [121]:
df = pd.read_csv('data/interactions.csv')
df_users = pd.read_csv('data/users.csv')
df_items = pd.read_csv('data/items.csv')

In [122]:
df_users.head()

,user_id,age,sex
0,1,45_54,NaN
1,2,18_24,0.0
2,3,65_inf,0.0
3,4,18_24,0.0
4,5,35_44,0.0


In [123]:
df_items.head()

,id,title,genres,authors,year
0,128115,Ворон-челобитчик,"Зарубежные детские книги,Сказки,Зарубежная кла...",Михаил Салтыков-Щедрин,1886
1,210979,Скрипка Ротшильда,"Классическая проза,Литература 19 века,Русская ...",Антон Чехов,1894
2,95632,Испорченные дети,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1869
3,247906,Странный человек,"Пьесы и драматургия,Литература 19 века",Михаил Лермонтов,1831
4,294280,Господа ташкентцы,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1873


In [124]:
df.head()

,user_id,item_id,progress,rating,start_date
0,126706,14433,80,NaN,2018-01-01
1,127290,140952,58,NaN,2018-01-01
2,66991,198453,89,NaN,2018-01-01
3,46791,83486,23,5.0,2018-01-01
4,79313,188770,88,5.0,2018-01-01


In [125]:
df['start_date'] = pd.to_datetime(df['start_date'])

In [126]:
df.dtypes

user_id                int64
item_id                int64
progress               int64
rating               float64
start_date    datetime64[ns]
dtype: object

In [127]:
duplicates = df.duplicated(subset=['user_id', 'item_id'], keep=False)
df_duplicates = df[duplicates].sort_values(by=['user_id', 'start_date'])
df = df[~duplicates]

In [128]:
df.shape

(1532918, 5)

In [129]:
df_duplicates = df_duplicates.groupby(['user_id', 'item_id']).agg({
    'progress': 'max',
    'rating': 'max',
    'start_date': 'min'
})
df = df.append(df_duplicates.reset_index(), ignore_index=True)

In [130]:
df['progress'] = df['progress'].astype(np.int8)
df['rating'] = df['rating'].astype(pd.SparseDtype(np.float32, np.nan))

In [131]:
df.to_pickle('data/interactions_preprocessed.pickle')

In [132]:
df_users['age'] = df_users['age'].astype('category')
df_users['sex'] = df_users['sex'].astype(pd.SparseDtype(np.float32, np.nan))

In [133]:
interaction_users = df['user_id'].unique()

common_users = len(np.intersect1d(interaction_users, df_users['user_id']))
users_only_in_interaction = len(np.setdiff1d(interaction_users, df_users['user_id']))
users_only_features = len(np.setdiff1d(df_users['user_id'], interaction_users))
total_users = common_users + users_only_in_interaction + users_only_features
print(f'Кол-во пользователей - {total_users}')
print(f'Кол-во пользователей c взаимодействиями и фичами - {common_users} ({common_users / total_users * 100:.2f}%)')
print(f'Кол-во пользователей только c взаимодействиями - {users_only_in_interaction} ({users_only_in_interaction / total_users * 100:.2f}%)')
print(f'Кол-во пользователей только c фичами - {users_only_features} ({users_only_features / total_users * 100:.2f}%)')


Кол-во пользователей - 158811
Кол-во пользователей c взаимодействиями и фичами - 135677 (85.43%)
Кол-во пользователей только c взаимодействиями - 15923 (10.03%)
Кол-во пользователей только c фичами - 7211 (4.54%)


In [134]:
df_users.to_pickle('data/users_preprocessed.pickle')

In [135]:
for col in ['genres', 'authors', 'year']:
    df_items[col] = df_items[col].astype('category')

In [136]:
interaction_items = df['item_id'].unique()

common_items = len(np.intersect1d(interaction_items, df_items['id']))
items_only_in_interaction = len(np.setdiff1d(interaction_items, df_items['id']))
items_only_features = len(np.setdiff1d(df_items['id'], interaction_items))
total_items = common_items + items_only_in_interaction + items_only_features
print(f'Кол-во книг - {total_items}')
print(f'Кол-во книг c взаимодействиями и фичами - {common_items} ({common_items / total_items * 100:.2f}%)')
print(f'Кол-во книг только c взаимодействиями - {items_only_in_interaction} ({items_only_in_interaction / total_items * 100:.2f}%)')
print(f'Кол-во книг только c фичами - {items_only_features} ({items_only_features / total_items * 100:.2f}%)')

Кол-во книг - 59599
Кол-во книг c взаимодействиями и фичами - 59599 (100.00%)
Кол-во книг только c взаимодействиями - 0 (0.00%)
Кол-во книг только c фичами - 0 (0.00%)


In [137]:
df_items.to_pickle('data/items_preprocessed.pickle')

# Метрики

In [138]:
df_true = pd.DataFrame({
    'user_id': ['Аня',                'Боря',               'Вася',         'Вася'],
    'item_id': ['Мастер и Маргарита', '451° по Фаренгейту', 'Зеленая миля', 'Рита Хейуорт и спасение из Шоушенка'],
})
df_true

,user_id,item_id
0,Аня,Мастер и Маргарита
1,Боря,451° по Фаренгейту
2,Вася,Зеленая миля
3,Вася,Рита Хейуорт и спасение из Шоушенка


Precision@k

Вначале посчитаем метрик для топ-2 (т.е. К = 2). Алгоритм следующий:

- Релевантные объекты, которые не были рекомендованы игнорируем (NaN)
- Определяем, какие релеватные рекомендации попали в топ-2 (hit)
- True positive для каждого пользователя
Делим TP на K
- Считаем Precision@K для каждого пользователя как сумму его TP/K
- Все Precision@K усредняем

In [139]:
df_recs = pd.DataFrame({
    'user_id': [
        'Аня', 'Аня', 'Аня', 
        'Боря', 'Боря', 'Боря', 
        'Вася', 'Вася', 'Вася',
    ],
    'item_id': [
        'Отверженные', 'Двенадцать стульев', 'Герои нашего времени', 
        '451° по Фаренгейту', '1984', 'О дивный новый мир',
        'Десять негритят', 'Искра жизни', 'Зеленая миля', 
    ],
    'rank': [
        1, 2, 3,
        1, 2, 3,
        1, 2, 3,
    ]
})
df_recs

,user_id,item_id,rank
0,Аня,Отверженные,1
1,Аня,Двенадцать стульев,2
2,Аня,Герои нашего времени,3
3,Боря,451° по Фаренгейту,1
4,Боря,1984,2
5,Боря,О дивный новый мир,3
6,Вася,Десять негритят,1
7,Вася,Искра жизни,2
8,Вася,Зеленая миля,3


In [140]:
df_merged = df_true.set_index(['user_id', 'item_id']).join(df_recs.set_index(['user_id', 'item_id']), how='left')
df_merged

rank
user_id item_id                                  
Аня     Мастер и Маргарита                    NaN
Боря    451° по Фаренгейту                    1.0
Вася    Зеленая миля                          3.0
        Рита Хейуорт и спасение из Шоушенка   NaN

In [141]:
df_merged['hit@2'] = df_merged['rank'] <= 2
df_merged

rank  hit@2
user_id item_id                                         
Аня     Мастер и Маргарита                    NaN  False
Боря    451° по Фаренгейту                    1.0   True
Вася    Зеленая миля                          3.0  False
        Рита Хейуорт и спасение из Шоушенка   NaN  False

In [142]:
df_merged['hit@2/2'] = df_merged['hit@2'] / 2
df_merged

rank  hit@2  hit@2/2
user_id item_id                                                  
Аня     Мастер и Маргарита                    NaN  False      0.0
Боря    451° по Фаренгейту                    1.0   True      0.5
Вася    Зеленая миля                          3.0  False      0.0
        Рита Хейуорт и спасение из Шоушенка   NaN  False      0.0

In [143]:
df_prec2 = df_merged.groupby(level=0)['hit@2/2'].sum()
df_prec2

user_id
Аня     0.0
Боря    0.5
Вася    0.0
Name: hit@2/2, dtype: float64

In [144]:


users_count = df_merged.index.get_level_values('user_id').nunique()
for k in [1, 2, 3]:
    hit_k = f'hit@{k}'
    df_merged[hit_k] = df_merged['rank'] <= k
    print(f'Precision@{k} = {(df_merged[hit_k] / k).sum() / users_count:.4f}')

Precision@1 = 0.3333
Precision@2 = 0.1667
Precision@3 = 0.2222


Recall@k

Посчитайте метрику полноты для k = 1, 2, 3

Recall@K (k =2) 

In [145]:
df_merged['users_item_count'] = df_merged.groupby(level='user_id')['rank'].transform(np.size)
df_merged

rank  hit@2  hit@2/2  hit@1  \
user_id item_id                                                            
Аня     Мастер и Маргарита                    NaN  False      0.0  False   
Боря    451° по Фаренгейту                    1.0   True      0.5   True   
Вася    Зеленая миля                          3.0  False      0.0  False   
        Рита Хейуорт и спасение из Шоушенка   NaN  False      0.0  False   

                                             hit@3  users_item_count  
user_id item_id                                                       
Аня     Мастер и Маргарита                   False               1.0  
Боря    451° по Фаренгейту                    True               1.0  
Вася    Зеленая миля                          True               2.0  
        Рита Хейуорт и спасение из Шоушенка  False               2.0

In [146]:
for k in [1, 2, 3]:
    hit_k = f'hit@{k}'
    # Уже посчитано
    # df_merged[hit_k] = df_merged['rank'] <= k  
    print(f"Recall@{k} = {(df_merged[hit_k] / df_merged['users_item_count']).sum() / users_count:.4f}")

Recall@1 = 0.3333
Recall@2 = 0.3333
Recall@3 = 0.5000


Почему Precision@k и Recall@k не всегда хорошо считать?

MRR

Эта метрика оценивает качество топ-N рекомендаций c учетом рангов/позиций. Основная идея - оценить "попадания" с весом, зависящим от позиции (обычно это обратная пропорциальная зависимость, то есть чем больше позиция, тем меньше вес).

**Mean Reciproal Rank, MRR**


$MRR \equiv \frac{1}{Q} \sum_{i=1}^Q\frac{1}{r_i}$

Где Q - это query или наш пользователь, а rank_i - позиция первой релевантной рекомендации

In [147]:
df_true = pd.DataFrame({
    'user_id': ['Аня',                'Боря',               'Вася',         'Вася'],
    'item_id': ['Мастер и Маргарита', '451° по Фаренгейту', 'Зеленая миля', 'Рита Хейуорт и спасение из Шоушенка'],
})
df_true

,user_id,item_id
0,Аня,Мастер и Маргарита
1,Боря,451° по Фаренгейту
2,Вася,Зеленая миля
3,Вася,Рита Хейуорт и спасение из Шоушенка


In [148]:
df_recs = pd.DataFrame({
    'user_id': [
        'Аня', 'Аня', 'Аня', 
        'Боря', 'Боря', 'Боря', 
        'Вася', 'Вася', 'Вася',
    ],
    'item_id': [
        'Отверженные', 'Двенадцать стульев', 'Герои нашего времени', 
        '451° по Фаренгейту', '1984', 'О дивный новый мир',
        'Десять негритят', 'Рита Хейуорт и спасение из Шоушенка', 'Зеленая миля', 
    ],
    'rank': [
        1, 2, 3,
        1, 2, 3,
        1, 2, 3,
    ]
})
df_recs

,user_id,item_id,rank
0,Аня,Отверженные,1
1,Аня,Двенадцать стульев,2
2,Аня,Герои нашего времени,3
3,Боря,451° по Фаренгейту,1
4,Боря,1984,2
5,Боря,О дивный новый мир,3
6,Вася,Десять негритят,1
7,Вася,Рита Хейуорт и спасение из Шоушенка,2
8,Вася,Зеленая миля,3


In [149]:
df_merged = df_true.set_index(['user_id', 'item_id']).join(df_recs.set_index(['user_id', 'item_id']), how='left')
df_merged = df_merged.sort_values(by=['user_id', 'rank'])
df_merged

rank
user_id item_id                                  
Аня     Мастер и Маргарита                    NaN
Боря    451° по Фаренгейту                    1.0
Вася    Рита Хейуорт и спасение из Шоушенка   2.0
        Зеленая миля                          3.0

In [150]:
df_merged['reciprocal_rank'] = 1 / df_merged['rank']
df_merged

rank  reciprocal_rank
user_id item_id                                                   
Аня     Мастер и Маргарита                    NaN              NaN
Боря    451° по Фаренгейту                    1.0         1.000000
Вася    Рита Хейуорт и спасение из Шоушенка   2.0         0.500000
        Зеленая миля                          3.0         0.333333

In [151]:
mrr = df_merged.groupby(level='user_id')['reciprocal_rank'].max()
mrr

user_id
Аня     NaN
Боря    1.0
Вася    0.5
Name: reciprocal_rank, dtype: float64

In [152]:
print(f"MRR = {mrr.fillna(0).mean()}")

MRR = 0.5


MAP

Написать код подсчета метрики MAP

In [153]:
df_merged = df_true.set_index(['user_id', 'item_id']).join(df_recs.set_index(['user_id', 'item_id']), how='left')
df_merged = df_merged.sort_values(by=['user_id', 'rank'])
df_merged

rank
user_id item_id                                  
Аня     Мастер и Маргарита                    NaN
Боря    451° по Фаренгейту                    1.0
Вася    Рита Хейуорт и спасение из Шоушенка   2.0
        Зеленая миля                          3.0

In [154]:
df_merged['cumulative_rank'] = df_merged.groupby(level='user_id').cumcount() + 1

In [155]:
df_merged

rank  cumulative_rank
user_id item_id                                                   
Аня     Мастер и Маргарита                    NaN                1
Боря    451° по Фаренгейту                    1.0                1
Вася    Рита Хейуорт и спасение из Шоушенка   2.0                1
        Зеленая миля                          3.0                2

In [156]:
df_merged['cumulative_rank'] = df_merged['cumulative_rank'] / df_merged['rank']

In [157]:
df_merged

rank  cumulative_rank
user_id item_id                                                   
Аня     Мастер и Маргарита                    NaN              NaN
Боря    451° по Фаренгейту                    1.0         1.000000
Вася    Рита Хейуорт и спасение из Шоушенка   2.0         0.500000
        Зеленая миля                          3.0         0.666667

In [158]:
df_merged['users_item_count'] = df_merged.groupby(level='user_id')['rank'].transform(np.size)
df_merged

rank  cumulative_rank  \
user_id item_id                                                      
Аня     Мастер и Маргарита                    NaN              NaN   
Боря    451° по Фаренгейту                    1.0         1.000000   
Вася    Рита Хейуорт и спасение из Шоушенка   2.0         0.500000   
        Зеленая миля                          3.0         0.666667   

                                             users_item_count  
user_id item_id                                                
Аня     Мастер и Маргарита                                1.0  
Боря    451° по Фаренгейту                                1.0  
Вася    Рита Хейуорт и спасение из Шоушенка               2.0  
        Зеленая миля                                      2.0

In [159]:
users_count = df_merged.index.get_level_values('user_id').nunique()

In [160]:
users_count

3

In [161]:
map3 = (df_merged["cumulative_rank"] / df_merged["users_item_count"]).sum() / users_count
print(f"MAP@3 = {map3}")

MAP@3 = 0.5277777777777778


Посчитаем все на нашем датасете

In [162]:
test_dates = df['start_date'].unique()[-7:]
test_dates = list(pairwise(test_dates))
test_dates

[(numpy.datetime64('2019-12-25T00:00:00.000000000'),
  numpy.datetime64('2019-12-26T00:00:00.000000000')),
 (numpy.datetime64('2019-12-26T00:00:00.000000000'),
  numpy.datetime64('2019-12-27T00:00:00.000000000')),
 (numpy.datetime64('2019-12-27T00:00:00.000000000'),
  numpy.datetime64('2019-12-28T00:00:00.000000000')),
 (numpy.datetime64('2019-12-28T00:00:00.000000000'),
  numpy.datetime64('2019-12-29T00:00:00.000000000')),
 (numpy.datetime64('2019-12-29T00:00:00.000000000'),
  numpy.datetime64('2019-12-30T00:00:00.000000000')),
 (numpy.datetime64('2019-12-30T00:00:00.000000000'),
  numpy.datetime64('2019-12-31T00:00:00.000000000'))]

In [163]:
split_dates = test_dates[0]
train = df[df['start_date'] < split_dates[0]]
test = df[(df['start_date'] >= split_dates[0]) & (df['start_date'] < split_dates[1])]
test = test[(test['rating'] >= 4) | (test['rating'].isnull())]
split_dates, train.shape, test.shape

((numpy.datetime64('2019-12-25T00:00:00.000000000'),
  numpy.datetime64('2019-12-26T00:00:00.000000000')),
 (1517994, 5),
 (2114, 5))

In [164]:
class PopularRecommender():
    def __init__(self, max_K=100, days=30, item_column='item_id', dt_column='date'):
        self.max_K = max_K
        self.days = days
        self.item_column = item_column
        self.dt_column = dt_column
        self.recommendations = []
        
    def fit(self, df, ):
        min_date = df[self.dt_column].max().normalize() - pd.DateOffset(days=self.days)
        self.recommendations = df.loc[df[self.dt_column] > min_date, self.item_column].value_counts().head(self.max_K).index.values
    
    def recommend(self, users=None, N=10):
        recs = self.recommendations[:N]
        if users is None:
            return recs
        else:
            return list(islice(cycle([recs]), len(users)))

In [165]:
pop_model = PopularRecommender(days=7, dt_column='start_date')
pop_model.fit(train)

In [166]:
top10_recs = pop_model.recommend()

In [167]:
item_titles = pd.Series(df_items['title'].values, index=df_items['id']).to_dict()
item_titles[128115]

'Ворон-челобитчик'

In [168]:
list(map(item_titles.get, top10_recs))

['Пикник на обочине',
 'Кавказский пленник',
 'Записки юного врача',
 'Богатый папа, бедный папа. Роберт Кийосаки (обзор)',
 'Москва и москвичи',
 'Понедельник начинается в субботу',
 'Хитрость',
 'Сказка о попе и о работнике его Балде',
 'Лорд, который влюбился. Тайный жених',
 'История государства Российского. Том 2. От Великого князя Святополка до Великого князя Мстислава Изяславовича']

In [169]:
recs = pd.DataFrame({'user_id': test['user_id'].unique()})
top_N = 10
recs['item_id'] = pop_model.recommend(recs['user_id'], N=top_N)
recs.head()

,user_id,item_id
0,38753,"[235407, 230067, 35265, 281005, 147734, 208935..."
1,101642,"[235407, 230067, 35265, 281005, 147734, 208935..."
2,13548,"[235407, 230067, 35265, 281005, 147734, 208935..."
3,130425,"[235407, 230067, 35265, 281005, 147734, 208935..."
4,93986,"[235407, 230067, 35265, 281005, 147734, 208935..."


In [170]:
recs = recs.explode('item_id')
recs.head(top_N + 2)

,user_id,item_id
0,38753,235407
0,38753,230067
0,38753,35265
0,38753,281005
0,38753,147734
0,38753,208935
0,38753,285394
0,38753,96052
0,38753,62715
0,38753,151190


In [171]:
recs['rank'] = recs.groupby('user_id').cumcount() + 1
recs.head(top_N + 2)

,user_id,item_id,rank
0,38753,235407,1
0,38753,230067,2
0,38753,35265,3
0,38753,281005,4
0,38753,147734,5
0,38753,208935,6
0,38753,285394,7
0,38753,96052,8
0,38753,62715,9
0,38753,151190,10


In [172]:
recs.dtypes

user_id     int64
item_id    object
rank        int64
dtype: object

In [173]:
recs['item_id'] = recs['item_id'].astype(np.int64)

In [174]:
test_recs = test.merge(recs, on = ['user_id', 'item_id'], how = 'inner')

In [175]:
test_recs.head()

,user_id,item_id,progress,rating,start_date,rank
0,102560,147734,0,NaN,2019-12-25,5
1,33152,147734,0,NaN,2019-12-25,5
2,55393,147734,1,NaN,2019-12-25,5
3,129781,147734,1,NaN,2019-12-25,5
4,57724,230067,74,NaN,2019-12-25,2


In [176]:
test_recs = test_recs.sort_values(by=['user_id', 'rank'])
test_recs.head()

,user_id,item_id,progress,rating,start_date,rank
46,474,235407,100,5.0,2019-12-25,1
18,1672,230067,12,NaN,2019-12-25,2
25,2345,208935,76,NaN,2019-12-25,6
61,9279,96052,100,NaN,2019-12-25,8
52,10260,35265,0,NaN,2019-12-25,3


### NDCG

Посчитайте метрику ndcg

In [177]:
def ndcg_at_k(
    df: pd.DataFrame,
    user_col: str = 'user_id',
    rank_col: str = 'item_id',
    k: int = 1,
    log_base: int = 2,
) -> pd.Series:
    dcg = ((df[rank_col] <= k).astype(int) / log_at_base(df[rank_col] + 1, log_base))
    idcg = (1 / log_at_base(np.arange(1, k + 1) + 1, log_base)).sum()
    res = (
        pd.DataFrame({user_col: df[user_col], "ndcg": dcg / idcg})
        .groupby(user_col)["ndcg"]
        .sum()
    )
    return res

# ItemToItem модели

### Нужно запрограммировать BM25Okapi

In [182]:
import math

In [183]:

class BM25:
    def __init__(self, corpus, tokenizer=None):
        self.corpus_size = len(corpus)
        self.avgdl = 0
        self.doc_freqs = []
        self.idf = {}
        self.doc_len = []
        self.tokenizer = tokenizer

        if tokenizer:
            corpus = self._tokenize_corpus(corpus)

        nd = self._initialize(corpus)
        self._calc_idf(nd)

    def _initialize(self, corpus):
        nd = {}  # word -> number of documents with word
        num_doc = 0
        for document in corpus:
            self.doc_len.append(len(document))
            num_doc += len(document)

            frequencies = {}
            for word in document:
                if word not in frequencies:
                    frequencies[word] = 0
                frequencies[word] += 1
            self.doc_freqs.append(frequencies)

            for word, freq in frequencies.items():
                try:
                    nd[word]+=1
                except KeyError:
                    nd[word] = 1

        self.avgdl = num_doc / self.corpus_size
        return nd

    def _tokenize_corpus(self, corpus):
        pool = Pool(cpu_count())
        tokenized_corpus = pool.map(self.tokenizer, corpus)
        return tokenized_corpus

    def _calc_idf(self, nd):
        raise NotImplementedError()

    def get_scores(self, query):
        raise NotImplementedError()

    def get_batch_scores(self, query, doc_ids):
        raise NotImplementedError()

    def get_top_n(self, query, documents, n=5):

        assert self.corpus_size == len(documents), "The documents given don't match the index corpus!"

        scores = self.get_scores(query)
        top_n = np.argsort(scores)[::-1][:n]
        return [documents[i] for i in top_n]



class BM25Okapi(BM25):
    def __init__(self, corpus, tokenizer=None, k1=1.5, b=0.75, epsilon=0.25):
        self.k1 = k1
        self.b = b
        self.epsilon = epsilon
        super().__init__(corpus, tokenizer)

    def _calc_idf(self, nd):
        """
        Calculates frequencies of terms in documents and in corpus.
        This algorithm sets a floor on the idf values to eps * average_idf
        """
        # collect idf sum to calculate an average idf for epsilon value
        idf_sum = 0
        # collect words with negative idf to set them a special epsilon value.
        # idf can be negative if word is contained in more than half of documents
        negative_idfs = []
        for word, freq in nd.items():
            idf = math.log(self.corpus_size - freq + 0.5) - math.log(freq + 0.5)
            self.idf[word] = idf
            idf_sum += idf
            if idf < 0:
                negative_idfs.append(word)
        self.average_idf = idf_sum / len(self.idf)

        eps = self.epsilon * self.average_idf
        for word in negative_idfs:
            self.idf[word] = eps

    def get_scores(self, query):
        """
        The ATIRE BM25 variant uses an idf function which uses a log(idf) score. To prevent negative idf scores,
        this algorithm also adds a floor to the idf value of epsilon.
        See [Trotman, A., X. Jia, M. Crane, Towards an Efficient and Effective Search Engine] for more info
        :param query:
        :return:
        """
        score = np.zeros(self.corpus_size)
        doc_len = np.array(self.doc_len)
        for q in query:
            q_freq = np.array([(doc.get(q) or 0) for doc in self.doc_freqs])
            score += (self.idf.get(q) or 0) * (q_freq * (self.k1 + 1) /
                                               (q_freq + self.k1 * (1 - self.b + self.b * doc_len / self.avgdl)))
        return score

    def get_batch_scores(self, query, doc_ids):
        """
        Calculate bm25 scores between query and subset of all docs
        """
        assert all(di < len(self.doc_freqs) for di in doc_ids)
        score = np.zeros(len(doc_ids))
        doc_len = np.array(self.doc_len)[doc_ids]
        for q in query:
            q_freq = np.array([(self.doc_freqs[di].get(q) or 0) for di in doc_ids])
            score += (self.idf.get(q) or 0) * (q_freq * (self.k1 + 1) /
                                               (q_freq + self.k1 * (1 - self.b + self.b * doc_len / self.avgdl)))
        return score.tolist()


class BM25Plus(BM25):
    def __init__(self, corpus, tokenizer=None, k1=1.5, b=0.75, delta=1):
        # Algorithm specific parameters
        self.k1 = k1
        self.b = b
        self.delta = delta
        super().__init__(corpus, tokenizer)

    def _calc_idf(self, nd):
        """
        -
        """


    def get_scores(self, query):
        """-"""

    def get_batch_scores(self, query, doc_ids):
        """
        Calculate bm25 scores between query and subset of all docs
        """
        assert all(di < len(self.doc_freqs) for di in doc_ids)




In [184]:
corpus = [
    "Hello there good man!",
    "It is quite windy in London",
    "How is the weather today?"
]

tokenized_corpus = [doc.split(" ") for doc in corpus]

bm25 = BM25Okapi(tokenized_corpus)

In [185]:
query = "windy London"
tokenized_query = query.split(" ")

doc_scores = bm25.get_scores(tokenized_query)



In [186]:
bm25.get_top_n(tokenized_query, corpus, n=1)

['It is quite windy in London']

In [187]:
query = "How London"
tokenized_query = query.split(" ")

doc_scores = bm25.get_scores(tokenized_query)

bm25.get_top_n(tokenized_query, corpus, n=1)

['How is the weather today?']

Возьмем BM25 из Implicit для рекомендаий

In [188]:
import os
import numpy as np 
import pandas as pd 
from itertools import islice, cycle
from more_itertools import pairwise
import scipy.sparse as sp
from itertools import islice, cycle

In [189]:
df = pd.read_pickle('data/interactions_preprocessed.pickle')
df_users = pd.read_pickle('data/users_preprocessed.pickle')
df_items = pd.read_pickle('data/items_preprocessed.pickle')

In [190]:
df_users.head()

,user_id,age,sex
0,1,45_54,NaN
1,2,18_24,0.0
2,3,65_inf,0.0
3,4,18_24,0.0
4,5,35_44,0.0


In [191]:
df_items.head()

,id,title,genres,authors,year
0,128115,Ворон-челобитчик,"Зарубежные детские книги,Сказки,Зарубежная кла...",Михаил Салтыков-Щедрин,1886
1,210979,Скрипка Ротшильда,"Классическая проза,Литература 19 века,Русская ...",Антон Чехов,1894
2,95632,Испорченные дети,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1869
3,247906,Странный человек,"Пьесы и драматургия,Литература 19 века",Михаил Лермонтов,1831
4,294280,Господа ташкентцы,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1873


In [192]:
df.dtypes

user_id                      int64
item_id                      int64
progress                      int8
rating        Sparse[float32, nan]
start_date          datetime64[ns]
dtype: object

In [193]:
df.shape, df_users.shape, df_items.shape

((1532998, 5), (142888, 3), (59599, 5))

In [194]:
users_inv_mapping = dict(enumerate(df['user_id'].unique()))
users_mapping = {v: k for k, v in users_inv_mapping.items()}
len(users_mapping)

151600

In [195]:
items_inv_mapping = dict(enumerate(df['item_id'].unique()))
items_mapping = {v: k for k, v in items_inv_mapping.items()}
len(items_mapping)

59599

In [196]:
df_items['title'] = df_items['title'].str.lower()
item_titles = pd.Series(df_items['title'].values, index=df_items['id']).to_dict()
len(item_titles), item_titles[128115]

(59599, 'ворон-челобитчик')

In [197]:
title_items = df_items.groupby('title')['id'].agg(list)
title_items

title
# 20 восьмая                                                     [201623]
# duo                                                             [72582]
# me too. роман                                                  [171172]
# партия                                                         [224512]
#1917: человек из раньшего времени. библиотека «проекта 1917»    [230768]
                                                                   ...   
…чума на оба ваши дома!                                          [226481]
№ 12, или история одного прекрасного юноши                        [20979]
伦巴德人的故事                                                          [119226]
地球への旅                                                            [148400]
�baby blues�                                                      [98635]
Name: id, Length: 57289, dtype: object

In [198]:
title_count = title_items.map(len)
title_count.value_counts()

1     55708
2      1197
3       245
4        71
5        38
6        11
7         8
8         3
9         2
23        1
18        1
47        1
13        1
12        1
11        1
Name: id, dtype: int64

In [199]:
df['rating'] = np.array(df['rating'].values, dtype=np.float32)

df.loc[df['item_id'].isin([44681, 162716])].groupby('item_id').agg({
    'progress': np.size,
    'rating': ['mean'],
    'start_date': ['min', 'max'],
})

progress rating start_date           
            size   mean        min        max
item_id                                      
44681        353   4.56 2018-01-24 2019-12-20
162716        59   4.80 2018-01-25 2019-12-30

In [200]:
last_date = df['start_date'].max().normalize()
folds = 7
start_date = last_date - pd.Timedelta(days=folds)
start_date, last_date

(Timestamp('2019-12-24 00:00:00'), Timestamp('2019-12-31 00:00:00'))

In [201]:
class TimeRangeSplit():
    """
        https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.date_range.html
    """
    def __init__(self, 
                 start_date, 
                 end_date=None, 
                 freq='D', 
                 periods=None, 
                 tz=None, 
                 normalize=False, 
                 closed=None, 
                 train_min_date=None,
                 filter_cold_users=True, 
                 filter_cold_items=True, 
                 filter_already_seen=True):
        
        self.start_date = start_date
        if end_date is None and periods is None:
            raise ValueError("Either 'end_date' or 'periods' must be non-zero, not both at the same time.")

        self.end_date = end_date
        self.freq = freq
        self.periods = periods
        self.tz = tz
        self.normalize = normalize
        self.closed = closed
        self.train_min_date = pd.to_datetime(train_min_date, errors='raise')
        self.filter_cold_users = filter_cold_users
        self.filter_cold_items = filter_cold_items
        self.filter_already_seen = filter_already_seen

        self.date_range = pd.date_range(
            start=start_date, 
            end=end_date, 
            freq=freq, 
            periods=periods, 
            tz=tz, 
            normalize=normalize, 
            closed=closed)

        self.max_n_splits = max(0, len(self.date_range) - 1)
        if self.max_n_splits == 0:
            raise ValueError("Provided parametrs set an empty date range.") 

    def split(self, 
              df, 
              user_column='user_id',
              item_column='item_id',
              datetime_column='date',
              fold_stats=False):
        df_datetime = df[datetime_column]
        if self.train_min_date is not None:
            train_min_mask = df_datetime >= self.train_min_date
        else:
            train_min_mask = df_datetime.notnull()

        date_range = self.date_range[(self.date_range >= df_datetime.min()) & 
                                     (self.date_range <= df_datetime.max())]

        for start, end in pairwise(date_range):
            fold_info = {
                'Start date': start,
                'End date': end
            }
            train_mask = train_min_mask & (df_datetime < start)
            train_idx = df.index[train_mask]
            if fold_stats:
                fold_info['Train'] = len(train_idx)

            test_mask = (df_datetime >= start) & (df_datetime < end)
            test_idx = df.index[test_mask]
            
            if self.filter_cold_users:
                new = np.setdiff1d(
                    df.loc[test_idx, user_column].unique(), 
                    df.loc[train_idx, user_column].unique())
                new_idx = df.index[test_mask & df[user_column].isin(new)]
                test_idx = np.setdiff1d(test_idx, new_idx)
                test_mask = df.index.isin(test_idx)
                if fold_stats:
                    fold_info['New users'] = len(new)
                    fold_info['New users interactions'] = len(new_idx)

            if self.filter_cold_items:
                new = np.setdiff1d(
                    df.loc[test_idx, item_column].unique(), 
                    df.loc[train_idx, item_column].unique())
                new_idx = df.index[test_mask & df[item_column].isin(new)]
                test_idx = np.setdiff1d(test_idx, new_idx)
                test_mask = df.index.isin(test_idx)
                if fold_stats:
                    fold_info['New items'] = len(new)
                    fold_info['New items interactions'] = len(new_idx)

            if self.filter_already_seen:
                user_item = [user_column, item_column]
                train_pairs = df.loc[train_idx, user_item].set_index(user_item).index
                test_pairs = df.loc[test_idx, user_item].set_index(user_item).index
                intersection = train_pairs.intersection(test_pairs)
                test_idx = test_idx[~test_pairs.isin(intersection)]
                # test_mask = rd.df.index.isin(test_idx)
                if fold_stats:
                    fold_info['Known interactions'] = len(intersection)

            if fold_stats:
                fold_info['Test'] = len(test_idx)

            yield (train_idx, test_idx, fold_info)

    def get_n_splits(self, df, datetime_column='date'):
        df_datetime = df[datetime_column]
        if self.train_min_date is not None:
            df_datetime = df_datetime[df_datetime >= self.train_min_date]

        date_range = self.date_range[(self.date_range >= df_datetime.min()) & 
                                     (self.date_range <= df_datetime.max())]

        return max(0, len(date_range) - 1)


In [202]:
last_date = df['start_date'].max().normalize()
folds = 7
start_date = last_date - pd.Timedelta(days=folds)
start_date, last_date

(Timestamp('2019-12-24 00:00:00'), Timestamp('2019-12-31 00:00:00'))

In [203]:
cv = TimeRangeSplit(start_date=start_date, periods=folds+1)

cv.max_n_splits, cv.get_n_splits(df, datetime_column='start_date')

(7, 7)

In [204]:
folds_with_stats = list(cv.split(
    df, 
    user_column='user_id',
    item_column='item_id',
    datetime_column='start_date',
    fold_stats=True
))

folds_info_with_stats = pd.DataFrame([info for _, _, info in folds_with_stats])

In [205]:
fold_dates = [(info['Start date'], info['End date']) for _, _, info in folds_with_stats]
fold_dates


[(Timestamp('2019-12-24 00:00:00', freq='D'),
  Timestamp('2019-12-25 00:00:00', freq='D')),
 (Timestamp('2019-12-25 00:00:00', freq='D'),
  Timestamp('2019-12-26 00:00:00', freq='D')),
 (Timestamp('2019-12-26 00:00:00', freq='D'),
  Timestamp('2019-12-27 00:00:00', freq='D')),
 (Timestamp('2019-12-27 00:00:00', freq='D'),
  Timestamp('2019-12-28 00:00:00', freq='D')),
 (Timestamp('2019-12-28 00:00:00', freq='D'),
  Timestamp('2019-12-29 00:00:00', freq='D')),
 (Timestamp('2019-12-29 00:00:00', freq='D'),
  Timestamp('2019-12-30 00:00:00', freq='D')),
 (Timestamp('2019-12-30 00:00:00', freq='D'),
  Timestamp('2019-12-31 00:00:00', freq='D'))]

In [206]:
def compute_metrics(df_true, df_pred, top_N):
    result = {}
    test_recs = df_true.set_index(['user_id', 'item_id']).join(df_pred.set_index(['user_id', 'item_id']))
    test_recs = test_recs.sort_values(by=['user_id', 'rank'])

    test_recs['users_item_count'] = test_recs.groupby(level='user_id')['rank'].transform(np.size)
    test_recs['reciprocal_rank'] = (1 / test_recs['rank']).fillna(0)
    test_recs['cumulative_rank'] = test_recs.groupby(level='user_id').cumcount() + 1
    test_recs['cumulative_rank'] = test_recs['cumulative_rank'] / test_recs['rank']
    
    # код из прошлого заданния про метрики

In [207]:
train_idx, test_idx, info = folds_with_stats[0]

train = df.loc[train_idx]
test = df.loc[test_idx]
train.shape, test.shape

((1515946, 5), (2045, 5))

In [208]:
def get_coo_matrix(df, 
                   user_col='user_id', 
                   item_col='item_id', 
                   weight_col=None, 
                   users_mapping=users_mapping, 
                   items_mapping=items_mapping):
    if weight_col is None:
        weights = np.ones(len(df), dtype=np.float32)
    else:
        weights = df[weight_col].astype(np.float32)

    interaction_matrix = sp.coo_matrix((
        weights, 
        (
            df[user_col].map(users_mapping.get), 
            df[item_col].map(items_mapping.get)
        )
    ))
    return interaction_matrix

In [209]:
train_mat = get_coo_matrix(train).tocsr()
train_mat


<151589x59599 sparse matrix of type '<class 'numpy.float32'>'
	with 1515946 stored elements in Compressed Sparse Row format>

In [210]:
from implicit.nearest_neighbours import CosineRecommender, TFIDFRecommender, BM25Recommender

In [211]:
cosine_model = CosineRecommender(K=10)
cosine_model.fit(train_mat.T) # 

  0%|          | 0/59599 [00:00<?, ?it/s]

In [212]:
md = BM25Recommender()
md.fit(train_mat.T) # 

  0%|          | 0/59599 [00:00<?, ?it/s]

In [213]:
top_N = 10
user_id = test['user_id'].iloc[0]
row_id = users_mapping[user_id]
print(f'Рекомендации для пользователя {user_id}, номер строки - {row_id}')

Рекомендации для пользователя 139942, номер строки - 10346


In [214]:
recs = cosine_model.recommend(row_id, train_mat, N=top_N, filter_already_liked_items=True)
recs = pd.DataFrame(recs, columns=['col_id', 'similarity'])
recs

,col_id,similarity
0,4341,0.297014
1,7353,0.220847
2,36593,0.215622
3,3802,0.188025
4,51215,0.145095
5,49085,0.128586
6,37852,0.102340
7,7873,0.101929
8,46769,0.100504
9,56270,0.100504


In [215]:
recs['item_id'] = recs['col_id'].map(items_inv_mapping.get)
recs['title'] = recs['item_id'].map(item_titles.get)
recs

,col_id,similarity,item_id,title
0,4341,0.297014,193358,#охотник на волков
1,7353,0.220847,125586,меч предназначения
2,36593,0.215622,203882,#имя для лис
3,3802,0.188025,90986,кровь эльфов
4,51215,0.145095,146180,крещение руси
5,49085,0.128586,264849,сплав закона
6,37852,0.102340,238155,владычица озера
7,7873,0.101929,7094,крещение огнем
8,46769,0.100504,240226,отпущение без грехов
9,56270,0.100504,1728,«злой город»


# Алгоритмы ранжирования

In [216]:
import numpy as np
import pandas as pd
import lightgbm

In [217]:
df = pd.DataFrame({
    "query_id":[i for i in range(100) for j in range(10)],
    "var1":np.random.random(size=(1000,)),
    "var2":np.random.random(size=(1000,)),
    "var3":np.random.random(size=(1000,)),
    "relevance":list(np.random.permutation([0,0,0,0,0, 0,0,0,1,1]))*100
})

In [218]:
df.head()

,query_id,var1,var2,var3,relevance
0,0,0.976094,0.765124,0.212162,1
1,0,0.555062,0.770926,0.497641,0
2,0,0.205938,0.588306,0.657546,0
3,0,0.889201,0.024080,0.057544,0
4,0,0.230709,0.258356,0.569535,0


In [219]:
df.tail()

,query_id,var1,var2,var3,relevance
995,99,0.907768,0.812209,0.986575,0
996,99,0.665996,0.088729,0.314359,0
997,99,0.597066,0.503740,0.912827,0
998,99,0.597674,0.724895,0.608859,0
999,99,0.059865,0.597198,0.322693,1


In [220]:
train_df = df[:800]  # first 80%
validation_df = df[800:]  # remaining 20%

qids_train = train_df.groupby("query_id")["query_id"].count().to_numpy()
X_train = train_df.drop(["query_id", "relevance"], axis=1)
y_train = train_df["relevance"]

qids_validation = validation_df.groupby("query_id")["query_id"].count().to_numpy()
X_validation = validation_df.drop(["query_id", "relevance"], axis=1)
y_validation = validation_df["relevance"]

In [221]:
qids_train

array([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10])

In [222]:
qids_validation

array([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10])

In [223]:
X_validation.head()

,var1,var2,var3
800,0.630259,0.287464,0.442245
801,0.940949,0.368070,0.100051
802,0.336355,0.488581,0.827659
803,0.147971,0.569369,0.485435
804,0.226146,0.558810,0.678019


In [224]:
y_validation.head()

800    1
801    0
802    0
803    0
804    0
Name: relevance, dtype: int64

In [225]:
model = lightgbm.LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
)

In [226]:
model.fit(
    X=X_train,
    y=y_train,
    group=qids_train,
    eval_set=[(X_validation, y_validation)],
    eval_group=[qids_validation],
    eval_at=10,
    verbose=10,
)

[10]	valid_0's ndcg@10: 0.553677
[20]	valid_0's ndcg@10: 0.564565
[30]	valid_0's ndcg@10: 0.561193
[40]	valid_0's ndcg@10: 0.559373
[50]	valid_0's ndcg@10: 0.528935
[60]	valid_0's ndcg@10: 0.526285
[70]	valid_0's ndcg@10: 0.544127
[80]	valid_0's ndcg@10: 0.529699
[90]	valid_0's ndcg@10: 0.526115
[100]	valid_0's ndcg@10: 0.532764


LGBMRanker(metric='ndcg', objective='lambdarank')

### Ranking problem

Введем обозначения:
* $Q = \{q_1, \dots, q_n\}$ список запросов
* $D_q = \{d_{q1}, \dots, d_{qm}\}$ -- набор документов, полученных для группы $q$
* $L_q = \{l_{q1}, \dots, l_{qm}\}$ -- метки релевантности для объектов из набор $D_q$

Каждый обьект $d_{qi}$ представлен в векторном пространстве признаков, описывающих связи между группой и объектом.

Таким образом, каждая группа связана с набором объектов. Например, группа - это запрос, а объект - это документ, если мы ранжируем документы по поисковому запросу.

Цель состоит в том, чтобы изучить функцию ранжирования $ f = f (d_ {qi}) $, такую, чтобы ранжирование объектов $ d_ {qi} $ для всех групп из $ Q $ основывалось на их оценках $ x_ {qi} = f (d_ {qi}) $ максимально приближенных к идеальному рейтингу согласно $ l_ {qi} $.

### Ranking quality metrics:
* __Precision__
    $$ \mbox{P}=\frac{|\{\mbox{relevant docs}\}\cap\{\mbox{retrieved docs}\}|}{|\{\mbox{retrieved docs}\}|} $$
* __Recall__
    $$ \mbox{R}=\frac{|\{\mbox{relevant docs}\}\cap\{\mbox{retrieved docs}\}|}{|\{\mbox{relevant docs}\}|} $$
    
    Обозначение $ @ k $ означает, что метрика рассчитывается для первых $ k $ документов из списка ранжирования.

     Например, если 1,2,5,7,9 - это ранги релевантных документов (перечисления начинаются с номера 1 из десяти полученных, то $ P @ 5 $ будет $\frac{3}{5}$.

* __Mean average precision (MAP)__
    $$\frac{1}{|Q|}\sum_{q \in Q} \frac{1}{|\mbox{relevant docs in } D_q|} \sum_{k} P@k(q) \times rel(q, k) $$
    
    Где $rel(q, k)$ - метка релевантности документа на k-й позиции в нашем рейтинге $ D_q $. Этот показатель вычисляет среднюю точность для запроса, взвешенного с учетом релевантности документов, а затем вычисляет среднее значение между всеми запросами.
    
* __Discounted cumulative gain (DCG)__
    $$\sum_{k=1}^{mq} \frac{2 ^ {l_{qk}}}{\log_2(k+1)}$$
    
    Эта метрика учитывает поведение пользователя: внимание пользователя находится наверху, а затем нелинейно уменьшается до конца.
    
* __NDCG__ - нормализованный DCG = DCG $ ~ / ~ $ IDCG, где IDCG - максимально возможное значение DCG с заданным набором меток релевантности.

* __AverageGain__ - представляет среднее значение значений метки для объектов с определенными значениями верхней метки.
* __[PFound](https://tech.yandex.com/catboost/doc/dg/references/pfound-docpage/#pfound)__
    
Больше на wiki: https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)

Параметр $ @ k $ для каждой метрики можно указать через параметр метрики «top», например «NDCG: top = 10» будет означать NDCG @ 10.